In [1]:
from GLMM_LA_fixed import *
# from GLMM_GH import *

In [2]:
##################### Simulation with Penn ############################
import os
truth = np.array([-1.5,0.1,-0.5,-0.3,0.4,-0.2,-0.25,0.35,-0.1,0.5]).reshape(10, 1)
var_name = []
for i in range(10):
    var_name += ['X' + str(i+1)]
import sys

In [3]:
file_dir = '../Simulation_data_GLMM/Setting_2/'
f = '2Sites_500PatientsEachSite_LargeVar4_Dataset14.csv'
df = pd.read_csv(file_dir + f, index_col=0)
# Load data
data1 = np.array(df[df['Site_ID'] == 1][var_name])
data2 = np.array(df[df['Site_ID'] == 2][var_name])
data = [data1, data2]
out1 = np.array(df[df['Site_ID'] == 1]['y']).reshape(500,1)
out2 = np.array(df[df['Site_ID'] == 2]['y']).reshape(500,1)
out = [out1, out2]

# LA

In [36]:
# Loading required packages
from numpy.polynomial.hermite import hermgauss
from numpy.linalg import inv
import pandas as pd
import numpy as np
import scipy
import time
from scipy.stats import norm

var_name = []
for i in range(10):
    var_name += ['X' + str(i+1)]

def Pi(x, beta_0, mu):
    result = np.asarray((np.exp(x @ beta_0 + mu) / (1 + np.exp(x @ beta_0 + mu))))
    if np.exp(x @ beta_0 + mu).max() == np.inf:
        return np.nan_to_num(result, nan=1)
    else:
        return result

def g(x, y, mu, beta_0, tau=1):
    g = sum(y * np.log(Pi(x, beta_0, mu)) + (1 - y) * np.log(1 - Pi(x, beta_0, mu))) \
        + np.log((np.sqrt(2 * np.pi) * tau) ** (-1) * np.exp(-mu ** 2 / (2 * tau ** 2)))
    return g

def g_b(x, y, mu, beta_0, tau=1):
    return np.sum(x * y - x * Pi(x, beta_0, mu), axis=0)

def g_u(x, y, mu, beta_0, tau=1):
    return sum(y - Pi(x, beta_0, mu)) - mu / tau ** 2

def g_uu(x, y, mu, beta_0, tau=1):
    result = np.nan_to_num(- np.asarray((np.exp(x @ beta_0 + mu) / (1 + np.exp(x @ beta_0 + mu)) ** 2)), nan=0)
    return sum(result) - 1 / tau ** 2

def g_ub(x, y, mu, beta_0, tau=1):
    result = np.nan_to_num(np.asarray((np.exp(x @ beta_0 + mu) / (1 + np.exp(x @ beta_0 + mu)) ** 2)), nan=0)
    return np.sum(- x * result, axis=0)

def g_bb(x, y, mu, beta_0, tau=1):
    result = 0
    for i in range(len(y)):
        result += -np.asarray(x[i].reshape(x.shape[1], 1) @ x[i].reshape(1, x.shape[1]) \
                              * np.nan_to_num((np.exp(x[i] @ beta_0 + mu) / (1 + np.exp(x[i] @ beta_0 + mu)) ** 2),
                                              nan=0))
    return result

def g_uuu(x, y, mu, beta_0, tau=1):
    return sum(- np.nan_to_num(np.asarray((np.exp(x @ beta_0 + mu) * (np.exp(x @ beta_0 + mu) - 1) \
                                           / (1 + np.exp(x @ beta_0 + mu)) ** 3)), nan=0))

def g_uub(x, y, mu, beta_0, tau=1):
    return np.sum(- x * np.nan_to_num(np.asarray((np.exp(x @ beta_0 + mu) * (np.exp(x @ beta_0 + mu) - 1) \
                                                  / (1 + np.exp(x @ beta_0 + mu)) ** 3)), nan=0), axis=0)

def g_ubb(x, y, mu, beta_0, tau=1):
    result = 0
    for i in range(len(y)):
        result += -np.asarray(x[i].reshape(x.shape[1], 1) @ x[i].reshape(1, x.shape[1]) \
                              * np.nan_to_num((np.exp(x[i] @ beta_0 + mu) * (np.exp(x[i] @ beta_0 + mu) - 1) \
                                               / (1 + np.exp(x[i] @ beta_0 + mu)) ** 3), nan=0))
    return result

def g_uuuu(x, y, mu, beta_0, tau=1):
    result = sum(- np.nan_to_num((np.exp(x @ beta_0 + mu) * (np.exp(x @ beta_0 + mu) - 1) \
                                  / (1 + np.exp(x @ beta_0 + mu)) ** 3), nan=0) \
                 + np.nan_to_num((3 * np.exp(2 * (x @ beta_0 + mu)) * (np.exp(x @ beta_0 + mu) - 1) \
                                  / (1 + np.exp(x @ beta_0 + mu)) ** 4), nan=0) \
                 - np.nan_to_num((np.exp(2 * (x @ beta_0 + mu)) / (1 + np.exp(x @ beta_0 + mu)) ** 3), nan=0) \
                 )
    return result

def g_uuub(x, y, mu, beta_0, tau=1):
    result = np.sum(- x * np.asarray(np.nan_to_num((np.exp(x @ beta_0 + mu) * (np.exp(x @ beta_0 + mu) - 1) \
                                                    / (1 + np.exp(x @ beta_0 + mu)) ** 3), nan=0) \
                                     + np.nan_to_num(
        (3 * np.exp(2 * (x @ beta_0 + mu)) * (np.exp(x @ beta_0 + mu) - 1) \
         / (1 + np.exp(x @ beta_0 + mu)) ** 4), nan=0) \
                                     - np.nan_to_num(
        (np.exp(2 * (x @ beta_0 + mu)) / (1 + np.exp(x @ beta_0 + mu)) ** 3), nan=0))
                    , axis=0)
    return result

def g_uubb(x, y, mu, beta_0, tau=1):
    result = 0
    for i in range(len(y)):
        result += -np.asarray(x[i].reshape(x.shape[1], 1) @ x[i].reshape(1, x.shape[1]) \
                              * (np.nan_to_num((np.exp(x @ beta_0 + mu) * (np.exp(x @ beta_0 + mu) - 1) \
                                                / (1 + np.exp(x @ beta_0 + mu)) ** 3), nan=0) \
                                 + np.nan_to_num((3 * np.exp(2 * (x @ beta_0 + mu)) * (np.exp(x @ beta_0 + mu) - 1) \
                                                  / (1 + np.exp(x @ beta_0 + mu)) ** 4), nan=0) \
                                 - np.nan_to_num(
                    (np.exp(2 * (x @ beta_0 + mu)) / (1 + np.exp(x @ beta_0 + mu)) ** 3), nan=0)))

    return result

def omega(x, y, mu, beta_0, tau=1):
    return np.sqrt(-1 / g_uu(x, y, mu, beta_0))

def omega_b(x, y, mu, beta_0, tau=1):
    return 0.5 * omega(x, y, mu, beta_0, tau) ** 3 * (g_uuu(x, y, mu, beta_0, tau) * mu_b(x, y, mu, beta_0, tau) \
                                                      + g_uub(x, y, mu, beta_0, tau))

def mu_b(x, y, mu, beta_0, tau=1):
    return omega(x, y, mu, beta_0, tau) ** 2 * g_ub(x, y, mu, beta_0, tau)

def mu_bb(x, y, mu, beta_0, tau=1):
    result = omega(x, y, mu, beta_0, tau) ** 2 * (mu_b(x, y, mu, beta_0, tau).reshape(x.shape[1], 1) \
                                                  @ mu_b(x, y, mu, beta_0, tau).reshape(1, x.shape[1]) \
                                                  * g_uuu(x, y, mu, beta_0, tau) \
                                                  + 2 * mu_b(x, y, mu, beta_0, tau).reshape(x.shape[1], 1) \
                                                  @ g_uub(x, y, mu, beta_0, tau).reshape(1, x.shape[1]) \
                                                  + g_ubb(x, y, mu, beta_0, tau))
    return result

def omega_bb(x, y, mu, beta_0, tau=1):
    result = 3 / 4 * omega(x, y, mu, beta_0, tau) ** 5 * (mu_b(x, y, mu, beta_0, tau) * g_uuu(x, y, mu, beta_0, tau) \
                                                          + g_uub(x, y, mu, beta_0, tau)).reshape(x.shape[1], 1) \
             @ (mu_b(x, y, mu, beta_0, tau) * g_uuu(x, y, mu, beta_0, tau) \
                + g_uub(x, y, mu, beta_0, tau)).reshape(1, x.shape[1]) + 1 / 2 * omega(x, y, mu, beta_0, tau) ** 3 \
             * (mu_bb(x, y, mu, beta_0, tau) * g_uuu(x, y, mu, beta_0, tau) \
                + (mu_b(x, y, mu, beta_0, tau).reshape(x.shape[1], 1) \
                   @ mu_b(x, y, mu, beta_0, tau).reshape(1, x.shape[1]) * g_uuuu(x, y, mu, beta_0, tau)) \
                + mu_b(x, y, mu, beta_0, tau).reshape(x.shape[1], 1) @ g_uuub(x, y, mu, beta_0, tau).reshape(1,
                                                                                                             x.shape[
                                                                                                                 1]) \
                + mu_bb(x, y, mu, beta_0, tau)
                )
    return result

def l(x, y, mu, beta_0, tau=1):
    l = 0.5 * np.log(2 * np.pi) + np.log(omega(x, y, mu, beta_0, tau)) + g(x, y, mu, beta_0, tau)
    return l

def l_1(x, y, mu, beta_0, tau=1):
    l1 = omega_b(x, y, mu, beta_0, tau) / omega(x, y, mu, beta_0, tau) \
         + g_u(x, y, mu, beta_0, tau) * mu_b(x, y, mu, beta_0, tau) + g_b(x, y, mu, beta_0, tau)
    return l1

def l_2(x, y, mu, beta_0, tau=1):
    l2 = omega(x, y, mu, beta_0, tau) ** (-2) * (omega_bb(x, y, mu, beta_0, tau) * omega(x, y, mu, beta_0, tau) \
                                                 - omega_b(x, y, mu, beta_0, tau).reshape(x.shape[1], 1) \
                                                 @ omega_b(x, y, mu, beta_0, tau).reshape(1, x.shape[1])) \
         + mu_bb(x, y, mu, beta_0, tau) * g_u(x, y, mu, beta_0, tau) + mu_b(x, y, mu, beta_0, tau).reshape(
        x.shape[1], 1) \
         @ (mu_b(x, y, mu, beta_0, tau) * g_uu(x, y, mu, beta_0, tau) + g_ub(x, y, mu, beta_0, tau)).reshape(1,
                                                                                                             x.shape[
                                                                                                                 1]) \
         + mu_b(x, y, mu, beta_0, tau).reshape(x.shape[1], 1) @ g_ub(x, y, mu, beta_0, tau).reshape(1, x.shape[1]) \
         + g_bb(x, y, mu, beta_0, tau)
    return l2

def max_mu(x, y, mu, beta_0, tau=1, max_iter=100):
    for step in range(max_iter):
        #         print('Step: ', step, '\n')
        mu_new = mu - g_u(x, y, mu, beta_0, tau) / g_uu(x, y, mu, beta_0, tau)
        diff = mu_new - mu
        #         print(diff)
        if np.abs(diff) < 10 ** (-10):
            #             print(mu)
            break;
        mu = mu_new
    return mu


def mapping(x):
    return (x - min(x))/(max(x) - min(x))

# def loss(x, y, beta, mu):
#     return sum((mapping(Pi(x, beta, mu)) - y)**2)

# Definitions for LA
def LA(X, y):
    # Added regularization
    pre_score = -10**10
    for lam in np.arange(0, 11, 1):

        # Regression with regularization
        beta = np.repeat(0, 10).reshape(10, 1)
        mu = np.repeat(0.1, len(y))
        tau = 1

        start_time = time.time()
        # print('Initial beta:', beta, "\n")
        for step_mu in range(3):
            for i in range(len(mu)):
                mu[i] = max_mu(X[i], y[i], mu[i], beta, tau)
            for step in range(50):
                l1 = 0
                l2 = 0
                for i in range(len(mu)):
                    l1 += l_1(X[i], y[i], mu[i], beta, tau)
                    l2 += l_2(X[i], y[i], mu[i], beta, tau)
                l1 -= (2 * lam * beta.transpose())[0]
                l2 -= np.diag(np.repeat(2 * lam, 10))
                delta = l1 @ inv(l2)
                new_beta = beta - delta.reshape(10, 1)
                if max(np.abs(delta)) < 10 ** (-6):
                    break;
                if max(np.abs(delta)) > 10 ** (3):
                    break;
                beta = new_beta
                if True in np.isnan(beta):
                    break;
#                 print('Step ', step + 1, ':\n')
                # print('Beta:\n', beta, '\n')
                # print('Diff:\n', delta, '\n')
                # print('Lam:\n', lam, '\n')

            if True in np.isnan(beta):
                break;
        # print('Beta:\n', beta, '\n')
        # print("--- %s seconds ---" % (time.time() - start_time))
        score = 0
        for i in range(len(X)):
            score += l(X[i], y[i], mu[i], beta) - sum(lam * (beta) **2)
        if score > pre_score:
            optimized_beta = beta
            optimized_mu = mu
            optimized_lam = lam
            # reset
            pre_score = score
            print(score)

    return [optimized_beta, optimized_mu]

def output(X, beta, true_beta, t):
    X = np.concatenate(X)

    V = np.diagflat(Pi(X, beta, 0) * (1 - Pi(X, beta, 0)))

    SE = np.sqrt(np.diag(inv(np.transpose(X) @ V @ X))).reshape(10,1)

    Z = beta/SE

    P = 2 * norm.cdf(-1 * np.abs(Z))

    CI_025  = beta - 1.959964 * SE
    CI_975  = beta + 1.959964 * SE

    df = pd.DataFrame({'Truth': np.transpose(true_beta)[0], 'Coef': np.transpose(beta)[0], 'Std.Err': np.transpose(SE)[0],
                       'z': np.transpose(Z)[0], 'P-value': np.transpose(P)[0],
                       '[0.025': np.transpose(CI_025)[0], '0.975]': np.transpose(CI_975)[0],
                       'RunTime': t},
                      index = var_name)
    return df

In [37]:
start_time = time.time()
[beta, mu] = LA(data, out)
t = time.time() - start_time
t = time.time() - start_time
output(data, beta, truth, t)
print("--- %s seconds ---" % (time.time() - start_time))

<ipython-input-36-3cbc38a128e2>:23: RuntimeWarning: divide by zero encountered in log
  + np.log((np.sqrt(2 * np.pi) * tau) ** (-1) * np.exp(-mu ** 2 / (2 * tau ** 2)))
<ipython-input-36-3cbc38a128e2>:33: RuntimeWarning: overflow encountered in exp
  result = np.nan_to_num(- np.asarray((np.exp(x @ beta_0 + mu) / (1 + np.exp(x @ beta_0 + mu)) ** 2)), nan=0)
<ipython-input-36-3cbc38a128e2>:33: RuntimeWarning: overflow encountered in square
  result = np.nan_to_num(- np.asarray((np.exp(x @ beta_0 + mu) / (1 + np.exp(x @ beta_0 + mu)) ** 2)), nan=0)
<ipython-input-36-3cbc38a128e2>:33: RuntimeWarning: invalid value encountered in true_divide
  result = np.nan_to_num(- np.asarray((np.exp(x @ beta_0 + mu) / (1 + np.exp(x @ beta_0 + mu)) ** 2)), nan=0)
<ipython-input-36-3cbc38a128e2>:49: RuntimeWarning: overflow encountered in exp
  return sum(- np.nan_to_num(np.asarray((np.exp(x @ beta_0 + mu) * (np.exp(x @ beta_0 + mu) - 1) \
<ipython-input-36-3cbc38a128e2>:49: RuntimeWarning: overflow encou

<ipython-input-36-3cbc38a128e2>:22: RuntimeWarning: divide by zero encountered in log
  g = sum(y * np.log(Pi(x, beta_0, mu)) + (1 - y) * np.log(1 - Pi(x, beta_0, mu))) \
<ipython-input-36-3cbc38a128e2>:22: RuntimeWarning: invalid value encountered in multiply
  g = sum(y * np.log(Pi(x, beta_0, mu)) + (1 - y) * np.log(1 - Pi(x, beta_0, mu))) \


[-432.5043285]
--- 65.84345293045044 seconds ---


In [39]:
output(data, beta, truth, t)

,Truth,Coef,Std.Err,z,P-value,[0.025,0.975],RunTime
X1,-1.50,-0.486869,0.109131,-4.461329,0.000008,-0.700762,-0.272977,65.840738
X2,0.10,-0.281353,0.227106,-1.238863,0.215396,-0.726472,0.163766,65.840738
X3,-0.50,-0.273817,0.152415,-1.796530,0.072410,-0.572545,0.024910,65.840738
X4,-0.30,-0.264319,0.139411,-1.895968,0.057964,-0.537560,0.008922,65.840738
X5,0.40,0.490233,0.144115,3.401688,0.000670,0.207774,0.772692,65.840738
X6,-0.20,-0.086230,0.073513,-1.172995,0.240798,-0.230312,0.057852,65.840738
X7,-0.25,-0.173790,0.047790,-3.636519,0.000276,-0.267457,-0.080123,65.840738
X8,0.35,0.295977,0.249265,1.187400,0.235070,-0.192573,0.784526,65.840738
X9,-0.10,0.523820,0.173856,3.012957,0.002587,0.183069,0.864571,65.840738
X10,0.50,0.457419,0.120504,3.795877,0.000147,0.221235,0.693603,65.840738


In [ ]:
output(data, beta, truth, t)

In [ ]:
start_time = time.time()
[beta, mu] = LA(data, out)
t = time.time() - start_time
t = time.time() - start_time
output(data, beta, truth, t)
print("--- %s seconds ---" % (time.time() - start_time))

# GH

In [ ]:
from GLMM_GH import *

In [ ]:
start_time = time.time()
[beta, mu] = GH(1, data, out)
t = time.time() - start_time
t = time.time() - start_time
output(data, beta, truth, t)
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
output(data, beta, truth, t)

In [ ]:
output(data, beta, truth, t)